## Votos parlamentares

Nesse notebook são calculados os votos dos parlamentares de São Paulo

In [29]:
import pandas as pd
from utils.load_csv import load_csv
from utils.save_csv import save_csv

In [30]:
deputados_eleitos = load_csv('deputados_eleitos.csv')

In [31]:
rmsp = ["ARUJÁ", "BARUERI", "BIRITIBA MIRIM", "CAIEIRAS", "CAJAMAR", "CARAPICUÍBA", "COTIA", "DIADEMA", "EMBU DAS ARTES", "EMBU-GUAÇU", "FERRAZ DE VASCONCELOS", "FRANCISCO MORATO",
        "FRANCO DA ROCHA", "GUARAREMA", "GUARULHOS", "ITAPECERICA DA SERRA", "ITAPEVI", "ITAQUAQUECETUBA", "JANDIRA", "JUQUITIBA", "MAIRIPORÃ", "MAUÁ", "MOGI DAS CRUZES", "OSASCO",
        "PIRAPORA DO BOM JESUS", "POÁ", "RIBEIRÃO PIRES", "RIO GRANDE DA SERRA", "SALESÓPOLIS", "SANTA ISABEL", "SANTANA DE PARNAÍBA", "SANTO ANDRÉ", "SÃO BERNARDO DO CAMPO",
        "SÃO CAETANO DO SUL", "SÃO LOURENÇO DA SERRA", "SÃO PAULO", "SUZANO", "TABOÃO DA SERRA", "VARGEM GRANDE PAULISTA"]

In [32]:
votos_total = deputados_eleitos.groupby("NM_CANDIDATO")["QT_VOTOS_NOMINAIS"].sum().rename("votos_total")

votos_capital = deputados_eleitos[deputados_eleitos["NM_MUNICIPIO"] == "SÃO PAULO"].groupby("NM_CANDIDATO")["QT_VOTOS_NOMINAIS"].sum().rename("votos_capital")

votos_rmsp = deputados_eleitos[deputados_eleitos["NM_MUNICIPIO"].isin(rmsp)].groupby("NM_CANDIDATO")["QT_VOTOS_NOMINAIS"].sum().rename("votos_rmsp")

votos_interior = (deputados_eleitos[~deputados_eleitos["NM_MUNICIPIO"].isin(rmsp)].groupby("NM_CANDIDATO")["QT_VOTOS_NOMINAIS"].sum().rename("votos_interior"))

votos = pd.concat([votos_total, votos_capital, votos_rmsp, votos_interior], axis = 1).fillna(0)

In [33]:
dados = []

for index, row in deputados_eleitos.drop_duplicates(subset = 'NM_CANDIDATO').iterrows():

    nome = row['NM_CANDIDATO']

    parlamentar = {
        'id_deputado': row['SQ_CANDIDATO'],
        'nome_civil': row['NM_CANDIDATO'],
        'nome_urna': row['NM_URNA_CANDIDATO'],
        #'cpf': row['cpf'],
        #'nascimento': row['nascimento'],
        'eleicao': row['DS_ELEICAO'],
        'cargo': row['DS_CARGO'],
        'partido': row['NM_PARTIDO'],
        'votos_total': int(votos.loc[nome, 'votos_total']) if nome in votos.index else 0,
        'votos_capital': int(votos.loc[nome, 'votos_capital']) if nome in votos.index else 0,
        'votos_rmsp': int(votos.loc[nome, 'votos_rmsp']) if nome in votos.index else 0,
        'votos_interior': int(votos.loc[nome, 'votos_interior']) if nome in votos.index else 0,
        'rep_capital': (int(votos.loc[nome, 'votos_capital']) / int(votos.loc[nome, 'votos_total']) * 100) if (nome in votos.index and votos.loc[nome, 'votos_total'] > 0) else 0.0,
        'rep_rmsp': (int(votos.loc[nome, 'votos_rmsp']) / int(votos.loc[nome, 'votos_total']) * 100) if (nome in votos.index and votos.loc[nome, 'votos_total'] > 0) else 0.0,
        'rep_interior': (int(votos.loc[nome, 'votos_interior']) / int(votos.loc[nome, 'votos_total']) * 100) if (nome in votos.index and votos.loc[nome, 'votos_total'] > 0) else 0.0,
        'ranking_total': votos['votos_total'].sort_values(ascending = False).reset_index().index[votos['votos_total'].sort_values(ascending = False).reset_index()['NM_CANDIDATO'] == nome][0] + 1 if nome in votos.index else None,
        'ranking_capital': votos['votos_capital'].sort_values(ascending = False).reset_index().index[votos['votos_capital'].sort_values(ascending = False).reset_index()['NM_CANDIDATO'] == nome][0] + 1 if nome in votos.index else None,
        'ranking_rmsp': votos['votos_rmsp'].sort_values(ascending = False).reset_index().index[votos['votos_rmsp'].sort_values(ascending = False).reset_index()['NM_CANDIDATO'] == nome][0] + 1 if nome in votos.index else None,
        'ranking_interior': votos['votos_interior'].sort_values(ascending = False).reset_index().index[votos['votos_interior'].sort_values(ascending = False).reset_index()['NM_CANDIDATO'] == nome][0] + 1 if nome in votos.index else None
    }

    dados.append(parlamentar)

dados_parlamentares = pd.DataFrame(dados)

In [34]:
dados_parlamentares

,id_deputado,nome_civil,nome_urna,eleicao,cargo,partido,votos_total,votos_capital,votos_rmsp,votos_interior,rep_capital,rep_rmsp,rep_interior,ranking_total,ranking_capital,ranking_rmsp,ranking_interior
0,250001611227,MARIO PALUMBO JUNIOR,DELEGADO PALUMBO,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Federal,Movimento Democrático Brasileiro,254898,141963,198606,56292,55.694042,77.915872,22.084128,13,8,8,94
1,250001611251,LUIZ FELIPE BALEIA TENUTO ROSSI,BALEIA ROSSI,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Federal,Movimento Democrático Brasileiro,236463,13936,23880,212583,5.893522,10.098832,89.901168,16,104,118,6
2,250001611254,FÁBIO EDUARDO DE OLIVEIRA TERUEL,FABIO TERUEL,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Federal,Movimento Democrático Brasileiro,235165,90826,155446,79719,38.622244,66.100823,33.899177,18,22,20,61
3,250001597775,LEONARDO DE SIQUEIRA LIMA,LEO SIQUEIRA,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Estadual,Partido Novo,90688,53542,65000,25688,59.039785,71.674312,28.325688,115,44,77,142
4,250001602101,RAFAEL SARAIVA GAIA,RAFAEL SARAIVA,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Estadual,UNIÃO BRASIL,98070,50651,65093,32977,51.647803,66.374019,33.625981,105,50,76,131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,250001609973,JORGE LUIS CARUSO,CARUSO,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Estadual,Movimento Democrático Brasileiro,82209,23771,34032,48177,28.915326,41.396927,58.603073,130,84,112,102
160,250001610018,ITAMAR FRANCISCO MACHADO BORGES,ITAMAR BORGES,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Estadual,Movimento Democrático Brasileiro,183480,13768,18981,164499,7.503815,10.344997,89.655003,38,105,122,10
161,250001620281,TABATA CLAUDIA AMARAL DE PONTES,TABATA AMARAL,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Federal,Partido Socialista Brasileiro,337873,143266,204416,133457,42.402323,60.500839,39.499161,7,7,7,17
162,250001620296,JONAS DONIZETTE FERREIRA,JONAS DONIZETTE,ELEIÇÕES GERAIS ESTADUAIS 2022,Deputado Federal,Partido Socialista Brasileiro,84044,8720,15791,68253,10.375518,18.788968,81.211032,127,117,126,73


In [35]:
save_csv(dados_parlamentares, 'dados_parlamentares.csv')

Base salva em data\dados_parlamentares.csv
